<a href="https://colab.research.google.com/github/PYDERT/DL-reproducability-Tracking-Objects-as-Points/blob/main/Tracking_Objects_as_Points_table_4_MOT17_Reproduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CenterTrack - Tracking Objects as Points
Simultaneous object detection and tracking using center points:
<img src="https://github.com/xingyizhou/CenterTrack/raw/master/readme/fig2.png">

**Original code :** [CenterTrack](https://github.com/xingyizhou/CenterTrack) by [Xingyi Zhou](https://github.com/xingyizhou)

**[Tracking Objects as Points](http://arxiv.org/abs/2004.01177)**,

Xingyi Zhou, Vladlen Koltun, Philipp Krähenbühl,

*arXiv technical report [(arXiv 2004.01177)](http://arxiv.org/abs/2004.01177)*

**Citation**
```
@article{zhou2020tracking,
  title={Tracking Objects as Points},
  author={Zhou, Xingyi and Koltun, Vladlen and Kr{\"a}henb{\"u}hl, Philipp},
  journal={ECCV},
  year={2020}
}
```

In [1]:
! nvidia-smi
!/usr/local/cuda/bin/nvcc --version

Tue Apr  5 18:11:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Installation
1. Install PyTorch:

In [2]:
import torch
print(torch.__version__)

1.10.0+cu111


In [3]:
! pip install torch==1.4.0 torchvision==0.5.0

     |████████████████████████████████| 753.4 MB 6.8 kB/s 
     |████████████████████████████████| 4.0 MB 26.8 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.


2. Install [COCOAPI](https://github.com/cocodataset/cocoapi):

In [4]:
! pip install cython; pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-78mqbojx
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-78mqbojx
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=264366 sha256=a158471283f411b6801ec370b4f93025eab76d49c770f7a0bc52ddb625e5d1f0
  Stored in directory: /tmp/pip-ephem-wheel-cache-sx3r8a5o/wheels/e2/6b/1d/344ac773c7495ea0b85eb228bc66daec7400a143a92d36b7b1
Successfully built pycocotools
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0.4
    Uninstalling pycocotools-2.0.4:
      Successfully uninstalled pycocotools-2.0.4


3. Clone this repo:

In [6]:
CenterTrack_ROOT="CenterTrack"
! git clone --recursive https://github.com/xingyizhou/CenterTrack $CenterTrack_ROOT
%cd CenterTrack

Cloning into 'CenterTrack'...
remote: Enumerating objects: 285, done.
remote: Total 285 (delta 0), reused 0 (delta 0), pack-reused 285
Receiving objects: 100% (285/285), 15.01 MiB | 20.43 MiB/s, done.
Resolving deltas: 100% (113/113), done.
/content/CenterTrack


You can manually install the [submodules](https://github.com/xingyizhou/CenterTrack/blob/master/.gitmodules) if you forget --recursive.

4. Install the requirements

In [7]:
! pip install -r requirements.txt
#! apt-get install libx264-dev

     |████████████████████████████████| 312 kB 5.5 MB/s 
     |████████████████████████████████| 151 kB 23.6 MB/s 
     |████████████████████████████████| 7.1 MB 2.8 MB/s 
     |████████████████████████████████| 87 kB 3.1 MB/s 
     |████████████████████████████████| 106 kB 40.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 64 kB 2.5 MB/s 
     |████████████████████████████████| 50 kB 5.3 MB/s 
     |████████████████████████████████| 42 kB 714 kB/s 
     |████████████████████████████████| 69 kB 7.1 MB/s 
     |████████████████████████████████| 297 kB 45.0 MB/s 
     |████████████████████████████████| 99 kB 1.2 MB/s 
     |████████████████████████████████| 297 kB 40.3 MB/s 
     |████████████████████████████████| 296 kB 42.8 MB/s 
     |████████████████████████████████| 296 kB 40.6 MB/s 
     |████████████████████████████████| 280 kB 45.2 MB/s 
     |████████████

In [8]:
%ls src/lib/model/networks/

backbones/     dla.py    generic_network.py  resdcn.py
base_model.py  dlav0.py  necks/              resnet.py


In [9]:
#%rm -r src/lib/model/networks/DCNv2
%ls

experiments/  NOTICE   README.md         src/
LICENSE       readme/  requirements.txt  videos/


5. Compile deformable convolutional (from DCNv2).

In [10]:
# clone if it is not automatically downloaded by `--recursive`.
%%shell

cd src/lib/model/networks/
git clone https://github.com/CharlesShang/DCNv2/  
cd DCNv2/
./make.sh

Cloning into 'DCNv2'...
remote: Enumerating objects: 214, done.
remote: Total 214 (delta 0), reused 0 (delta 0), pack-reused 214
Receiving objects: 100% (214/214), 1.42 MiB | 8.90 MiB/s, done.
Resolving deltas: 100% (121/121), done.
running build
running build_ext
building '_ext' extension
creating build
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/content
creating build/temp.linux-x86_64-3.7/content/CenterTrack
creating build/temp.linux-x86_64-3.7/content/CenterTrack/src
creating build/temp.linux-x86_64-3.7/content/CenterTrack/src/lib
creating build/temp.linux-x86_64-3.7/content/CenterTrack/src/lib/model
creating build/temp.linux-x86_64-3.7/content/CenterTrack/src/lib/model/networks
creating build/temp.linux-x86_64-3.7/content/CenterTrack/src/lib/model/networks/DCNv2
creating build/temp.linux-x86_64-3.7/content/CenterTrack/src/lib/model/networks/DCNv2/src
creating build/temp.linux-x86_64-3.7/content/CenterTrack/src/lib/model/networks/DCNv2/src/cpu
creating

6. Download pertained models for [monocular 3D tracking](https://drive.google.com/open?id=1e8zR1m1QMJne-Tjp-2iY_o81hn2CiQRt), [80-category tracking](https://drive.google.com/open?id=1tJCEJmdtYIh8VuN8CClGNws3YO7QGd40), or [pose tracking](https://drive.google.com/open?id=1tJCEJmdtYIh8VuN8CClGNws3YO7QGd40) and move them to 
`$CenterTrack_ROOT/models/`. 

  More models can be found in [Model zoo](https://github.com/xingyizhou/CenterTrack/blob/master/readme/MODEL_ZOO.md).

In [11]:
%mkdir models
! gdown --id 1e8zR1m1QMJne-Tjp-2iY_o81hn2CiQRt -O models/nuScenes_3Dtracking.pth
! gdown --id 1tJCEJmdtYIh8VuN8CClGNws3YO7QGd40 -O models/coco_tracking.pth
! gdown --id 1H0YvFYCOIZ06EzAkC2NxECNQGXxK27hH -O models/coco_pose_tracking.pth
! gdown --id 1sf1bWJ1LutwQ_wp176nd2Y3HII9WeFf0 -O models/mot17_half.pth
! gdown --id 1h_8Ts11rf0GQ4_n6FgmCeBuFcWrRjJfa -O models/mot17_fulltrain.pth
! gdown --id 1rIVl-jSG6oiBdiJmCvIAUOeasT7YllRZ -O models/crowdhuman.pth

Downloading...
From: https://drive.google.com/uc?id=1e8zR1m1QMJne-Tjp-2iY_o81hn2CiQRt
To: /content/CenterTrack/models/nuScenes_3Dtracking.pth
100% 81.8M/81.8M [00:00<00:00, 249MB/s]
Downloading...
From: https://drive.google.com/uc?id=1tJCEJmdtYIh8VuN8CClGNws3YO7QGd40
To: /content/CenterTrack/models/coco_tracking.pth
100% 79.5M/79.5M [00:00<00:00, 124MB/s]
Downloading...
From: https://drive.google.com/uc?id=1H0YvFYCOIZ06EzAkC2NxECNQGXxK27hH
To: /content/CenterTrack/models/coco_pose_tracking.pth
100% 81.2M/81.2M [00:00<00:00, 210MB/s]
Downloading...
From: https://drive.google.com/uc?id=1sf1bWJ1LutwQ_wp176nd2Y3HII9WeFf0
To: /content/CenterTrack/models/mot17_half.pth
100% 80.0M/80.0M [00:00<00:00, 201MB/s]
Downloading...
From: https://drive.google.com/uc?id=1h_8Ts11rf0GQ4_n6FgmCeBuFcWrRjJfa
To: /content/CenterTrack/models/mot17_fulltrain.pth
100% 80.0M/80.0M [00:00<00:00, 249MB/s]
Downloading...
From: https://drive.google.com/uc?id=1rIVl-jSG6oiBdiJmCvIAUOeasT7YllRZ
To: /content/CenterTrack

In [12]:
# %cd ../content/CenterTrack

In [25]:
%cd src/tools

/content/CenterTrack/src/tools


In [26]:
%ls

annot_bbox.py                  convert_nuScenes.py   nuScenes_lib/
convert_crowdhuman_to_coco.py  eval_kitti_track/     remove_optimizers.py
convert_kittitrack_to_coco.py  eval_motchallenge.py  vis_tracking_kitti.py
convert_mot_det_to_results.py  get_mot_17.sh         vis_tracking_mot.py
convert_mot_to_coco.py         _init_paths.py


In [16]:
# Before running this, make the data/mot17 folders. Mkdir not working

!bash get_mot_17.sh

Streaminguitvoer ingekort tot de laatste 5000 regels.
  inflating: MOT17/train/MOT17-04-DPM/img1/000496.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000482.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000509.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000521.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000247.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000253.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000535.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000284.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000290.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000079.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000045.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000723.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000737.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000051.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000904.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000910.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000086.

In [27]:
%ls

annot_bbox.py                  convert_nuScenes.py   nuScenes_lib/
convert_crowdhuman_to_coco.py  eval_kitti_track/     remove_optimizers.py
convert_kittitrack_to_coco.py  eval_motchallenge.py  vis_tracking_kitti.py
convert_mot_det_to_results.py  get_mot_17.sh         vis_tracking_mot.py
convert_mot_to_coco.py         _init_paths.py


In [28]:
# Before running this, move the train and test folders from data/mot17/MOT17 out into data/mot17 

!python convert_mot_to_coco.py

MOT17-02-FRCNN: 600 images
 600 ann images
MOT17-04-FRCNN: 1050 images
 1050 ann images
MOT17-05-FRCNN: 837 images
 837 ann images
MOT17-09-FRCNN: 525 images
 525 ann images
MOT17-10-FRCNN: 654 images
 654 ann images
MOT17-11-FRCNN: 900 images
 900 ann images
MOT17-13-FRCNN: 750 images
 750 ann images
loaded train_half for 2664 images and 45732 samples
MOT17-02-FRCNN: 600 images
 600 ann images
MOT17-04-FRCNN: 1050 images
 1050 ann images
MOT17-05-FRCNN: 837 images
 837 ann images
MOT17-09-FRCNN: 525 images
 525 ann images
MOT17-10-FRCNN: 654 images
 654 ann images
MOT17-11-FRCNN: 900 images
 900 ann images
MOT17-13-FRCNN: 750 images
 750 ann images
loaded val_half for 2652 images and 40096 samples
MOT17-02-FRCNN: 600 images
 600 ann images
MOT17-04-FRCNN: 1050 images
 1050 ann images
MOT17-05-FRCNN: 837 images
 837 ann images
MOT17-09-FRCNN: 525 images
 525 ann images
MOT17-10-FRCNN: 654 images
 654 ann images
MOT17-11-FRCNN: 900 images
 900 ann images
MOT17-13-FRCNN: 750 images
 750 

In [29]:
!python convert_mot_det_to_results.py

In [30]:
%cd ..

/content/CenterTrack/src


## Table 4 reproduction

Before running any tests:

1. Comment out lines 32 and 33 in src/lib/logger.py


Total model of authors results - MOTA 66.1%



In [38]:
!python test.py tracking --exp_id mot17_half --dataset mot --dataset_version 17halfval --pre_hm --ltrb_amodal --track_thresh 0.4 --pre_thresh 0.5 --load_model ../models/mot17_half.pth

/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
Running tracking
Using tracking threshold for out threshold! 0.4
Fix size testing.
training chunk_sizes: [32]
input h w: 544 960
heads {'hm': 1, 'reg': 2, 'wh': 2, 'tracking': 2, 'ltrb_amodal': 4}
weights {'hm': 1, 'reg': 1, 'wh': 0.1, 'tracking': 1, 'ltrb_amodal': 0.1}
head conv {'hm': [256], 'reg': [256], 'wh': [256], 'tracking': [256], 'ltrb_amodal': [256]}
Namespace(K=100, add_05=False, amodel_offset_weight=1, arch='dla_34', aug_rot=0, backbone='dla34', batch_size=32, chunk_sizes=[32], custom_dataset_ann_path='', custom_dataset_img_path='', data_dir='/content/CenterTrack/src/lib/../../data', dataset='mot', dataset_version='17halfval', debug=0, debug_dir='/content/CenterTrack/src/lib/../../exp/tracking/mot17_half/debug', debugger_theme='white'

Model with heatmap disabled - MOTA 63.4%

In [37]:
 !python test.py tracking --exp_id mot17_half --dataset mot --dataset_version 17halfval --ltrb_amodal --track_thresh 0.4 --pre_thresh 0.5 --load_model ../models/mot17_half.pth

/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
Running tracking
Using tracking threshold for out threshold! 0.4
Fix size testing.
training chunk_sizes: [32]
input h w: 544 960
heads {'hm': 1, 'reg': 2, 'wh': 2, 'tracking': 2, 'ltrb_amodal': 4}
weights {'hm': 1, 'reg': 1, 'wh': 0.1, 'tracking': 1, 'ltrb_amodal': 0.1}
head conv {'hm': [256], 'reg': [256], 'wh': [256], 'tracking': [256], 'ltrb_amodal': [256]}
Namespace(K=100, add_05=False, amodel_offset_weight=1, arch='dla_34', aug_rot=0, backbone='dla34', batch_size=32, chunk_sizes=[32], custom_dataset_ann_path='', custom_dataset_img_path='', data_dir='/content/CenterTrack/src/lib/../../data', dataset='mot', dataset_version='17halfval', debug=0, debug_dir='/content/CenterTrack/src/lib/../../exp/tracking/mot17_half/debug', debugger_theme='white'

Without offset - MOTA 65.8%

**Before running**

1. Paste the following line in src/lib/utils/tracker.py under line 34 and comment out line 34 for no tracking: 

        [det['ct'] + det['tracking'] for det in results], np.float32) # N x 2

In [ ]:
!python test.py tracking --exp_id mot17_half --dataset mot --dataset_version 17halfval --pre_hm --ltrb_amodal --track_thresh 0.4 --pre_thresh 0.5 --load_model ../models/mot17_half.pth

/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
Running tracking
Using tracking threshold for out threshold! 0.4
Fix size testing.
training chunk_sizes: [32]
input h w: 544 960
heads {'hm': 1, 'reg': 2, 'wh': 2, 'tracking': 2, 'ltrb_amodal': 4}
weights {'hm': 1, 'reg': 1, 'wh': 0.1, 'tracking': 1, 'ltrb_amodal': 0.1}
head conv {'hm': [256], 'reg': [256], 'wh': [256], 'tracking': [256], 'ltrb_amodal': [256]}
Namespace(K=100, add_05=False, amodel_offset_weight=1, arch='dla_34', aug_rot=0, backbone='dla34', batch_size=32, chunk_sizes=[32], custom_dataset_ann_path='', custom_dataset_img_path='', data_dir='/content/CenterTrack/src/lib/../../data', dataset='mot', dataset_version='17halfval', debug=0, debug_dir='/content/CenterTrack/src/lib/../../exp/tracking/mot17_half/debug', debugger_theme='white'

Tracking only - MOTA 63.1%

In [ ]:
 !python test.py tracking --exp_id mot17_half --dataset mot --dataset_version 17halfval --ltrb_amodal --track_thresh 0.4 --pre_thresh 0.5 --load_model ../models/mot17_half.pth

/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
Running tracking
Using tracking threshold for out threshold! 0.4
Fix size testing.
training chunk_sizes: [32]
input h w: 544 960
heads {'hm': 1, 'reg': 2, 'wh': 2, 'tracking': 2, 'ltrb_amodal': 4}
weights {'hm': 1, 'reg': 1, 'wh': 0.1, 'tracking': 1, 'ltrb_amodal': 0.1}
head conv {'hm': [256], 'reg': [256], 'wh': [256], 'tracking': [256], 'ltrb_amodal': [256]}
Namespace(K=100, add_05=False, amodel_offset_weight=1, arch='dla_34', aug_rot=0, backbone='dla34', batch_size=32, chunk_sizes=[32], custom_dataset_ann_path='', custom_dataset_img_path='', data_dir='/content/CenterTrack/src/lib/../../data', dataset='mot', dataset_version='17halfval', debug=0, debug_dir='/content/CenterTrack/src/lib/../../exp/tracking/mot17_half/debug', debugger_theme='white'